In [28]:
from QAOAUtils import *
from oputils import *
import pickle as pkl
import numpy as np

In [29]:
DATA = []
ws_list = ['GW2','GW3','BM2','BM3']
n = 10
rotation_options = list(range(n))+[None]

In [30]:
def TSP_QUBO(n,max_val,eps_list):
    points = np.random.random((n,2)) * max_val * 2 - max_val
    W_mat = np.array([[np.linalg.norm(points[i]-points[j]) for i in range(n)] for j in range(n)])

    A_list = [num_to_bin((1+eps)*np.max(W_mat),(n-1)**2) for eps in eps_list]
    reward = num_to_bin(0,(n-1)**2)
    for i in range(0,n-1):
        for j in range(0,n-1):
            for t in range(0,n-2):
                reward += num_to_bin(W_mat[i+1][j+1],(n-1)**2) * indexed_bin_op((n-1)*t+i,(n-1)**2) *  indexed_bin_op((n-1)*(t+1)+j,(n-1)**2)
        reward += num_to_bin(W_mat[0][i+1],(n-1)**2) * indexed_bin_op(i,(n-1)**2) 
        reward += num_to_bin(W_mat[0][i+1],(n-1)**2) *  indexed_bin_op((n-1)*(n-2)+i,(n-1)**2)
    
    penalty1=num_to_bin(0,(n-1)**2)
    for t in range(n-1):
        temp_op = num_to_bin(1,(n-1)**2)
        for i in range(n-1):
            temp_op -= indexed_bin_op((n-1)*t+i,(n-1)**2)
        penalty1+=temp_op**2
    
    penalty2=num_to_bin(0,(n-1)**2)
    for i in range(n-1):
        temp_op = num_to_bin(1,(n-1)**2)
        for t in range(n-1):
            temp_op -= indexed_bin_op((n-1)*t+i,(n-1)**2)
        penalty2+=temp_op**2
    
    cost = [reward+A*(penalty1+penalty2) for A in A_list]

    return points,W_mat, [bin_to_qubo(c) for c in cost]

In [31]:
def MIS_QUBO(n,A_list,graph='gnp'):
    if(graph == 'gnp'):
        G=nx.gnp_random_graph(n,0.25)
        return  [np.diag(np.ones(n))- (A/2) *nx.adjacency_matrix(G).toarray() for A in A_list]
    elif(graph=='nws'):
        G=nx.newman_watts_strogatz_graph(n,3,0.5)
        return [np.diag(np.ones(n))- (A/2) *nx.adjacency_matrix(G).toarray() for A in A_list]
    elif(graph=='regular'):
        G=nx.random_regular_graph(3,n)
        return [np.diag(np.ones(n))-(A/2) *nx.adjacency_matrix(G).toarray() for A in A_list]
    else:
        return "????"

In [37]:
def depth0_ws_comp(n,A_list,ws_list = ['BM2','BM3','GW2','GW3'],rotation_options = None):
    if(rotation_options is None):
        rotation_options = list(range(n))+[None]
    best_angle_data = {ws: {'max_cost':[],'max_probs':[]}  for ws in ws_list}
    comparison_data = {ws: {-1:{'cost':[],'probs':[]},0:{'cost':[],'probs':[]},None:{'cost':[],'probs':[]}}  for ws in ws_list} | {None:{'cost':[], 'probs':[]}}
    ws_data = []
    for _ in tqdm(range(len(A_list))):

        A = A_list[_]
        initial_data = initialize(A)
        precomp = initial_data[0]
        BM2_theta_list = initial_data[1]
        BM3_theta_list = initial_data[2]
        GW2_theta_list = initial_data[3]
        GW3_theta_list = initial_data[4]
        v = initial_data[5]
        M = initial_data[6]
        ws_data.append(initial_data)
        A_list.append(A)
        qc_data = {'BM2':{r: Q2_data(BM2_theta_list,r) for r in rotation_options},
        'BM3':{r:Q3_data(BM3_theta_list,r) for r in rotation_options},
        'GW2':{r:Q2_data(GW2_theta_list,r) for r in rotation_options},
        'GW3':{r:Q3_data(GW3_theta_list,r) for r in rotation_options}}
    
        opt_data = {ws:{'cost':[],'probs':[]} for ws in ws_list}
        
        for ws in ws_list:
            for r in rotation_options:
                opt_data[ws]['cost'].append(expval(precomp,QAOA_eval(precomp,[],mixer_ops=qc_data[ws][r][1],init=qc_data[ws][r][0])))
                opt_data[ws]['probs'].append(opt_sampling_prob(v,precomp,[],mixer_ops=qc_data[ws][r][1],init=qc_data[ws][r][0]))
            l = np.argwhere(opt_data[ws]['cost']==np.max(opt_data[ws]['cost']))
            l = np.reshape(l,(len(l),))
            best_angle_data[ws]['max_cost']+=[l.tolist()]
            l = np.argwhere(opt_data[ws]['probs']==np.max(opt_data[ws]['probs']))
            l = np.reshape(l,(len(l),))
            best_angle_data[ws]['max_probs']+=[l.tolist()]

            comparison_data[ws][-1]['cost'].append(opt_data[ws]['cost'][-2])
            comparison_data[ws][-1]['probs'].append(opt_data[ws]['probs'][-2])
            comparison_data[ws][None]['cost'].append(opt_data[ws]['cost'][-1])
            comparison_data[ws][None]['probs'].append(opt_data[ws]['probs'][-1])
            comparison_data[ws][0]['cost'].append(opt_data[ws]['cost'][0])
            comparison_data[ws][0]['probs'].append(opt_data[ws]['probs'][0])
            
            comparison_data[None]['cost'].append(expval(precomp,QAOA_eval(precomp,[])))
            comparison_data[None]['probs'].append(opt_sampling_prob(v,precomp,[]))
    return comparison_data,best_angle_data,ws_data,A_list

In [38]:
[dual_graph(x[-1]).shape for x in TSP_QUBO(4,1,np.linspace(0.1,1.1,25))[-1]]

[(10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10),
 (10, 10)]

In [55]:
A_list = reduce(lambda a,b: a+b, [[dual_graph(-x[-1]) for x in TSP_QUBO(4,1,np.linspace(0.1,1.1,5))[-1]] for i in range(20)])
comparison_data,best_angle_data,ws_data,A_list= depth0_ws_comp(n,A_list,ws_list = ws_list,rotation_options = rotation_options)
DATA+=[comparison_data,best_angle_data,ws_data,A_list]

  0%|          | 0/100 [00:00<?, ?it/s]

In [57]:
A_list = reduce(lambda a,b: a+b, [[dual_graph(x) for x in MIS_QUBO(9, np.linspace(1.1,2.1,5))] for i in range(20)])
comparison_data,best_angle_data,ws_data,A_list= depth0_ws_comp(n,A_list,ws_list = ws_list,rotation_options = rotation_options)
DATA+=[comparison_data,best_angle_data,ws_data,A_list]

  0%|          | 0/100 [00:00<?, ?it/s]

In [58]:
A_list = reduce(lambda a,b: a+b, [[dual_graph(x) for x in MIS_QUBO(9, np.linspace(1.1,2.1,5),'nws')] for i in range(20)])
comparison_data,best_angle_data,ws_data,A_list= depth0_ws_comp(n,A_list,ws_list = ws_list,rotation_options = rotation_options)
DATA+=[comparison_data,best_angle_data,ws_data,A_list]

  0%|          | 0/100 [00:00<?, ?it/s]

In [54]:
reduce(lambda a,b: a+b, [[dual_graph(x) for x in TSP_QUBO(4,1,np.linspace(1.1,2.1,5)) for i in range(20)])

100


In [71]:
l=[[np.array(np.linspace(0,95,20),dtype='int')+i] for i in range(5)]

In [72]:
p_data = {'TSP':DATA[0:4],'MIS-GNP':DATA[4:8],'MIS-NWS':DATA[8:12]}

In [73]:
{"TSP_"+str(i):[np.array(x)[l[i]] for x in p_data["TSP"]] for i in range(5)}

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [74]:
print(p_data["TSP"])

[{'GW2': {-1: {'cost': [], 'probs': []}, 0: {'cost': [], 'probs': []}, None: {'cost': [], 'probs': []}}, 'GW3': {-1: {'cost': [], 'probs': []}, 0: {'cost': [], 'probs': []}, None: {'cost': [], 'probs': []}}, 'BM2': {-1: {'cost': [], 'probs': []}, 0: {'cost': [], 'probs': []}, None: {'cost': [], 'probs': []}}, 'BM3': {-1: {'cost': [], 'probs': []}, 0: {'cost': [], 'probs': []}, None: {'cost': [], 'probs': []}}, None: {'cost': [], 'probs': []}}, {'GW2': {'max_cost': [], 'max_probs': []}, 'GW3': {'max_cost': [], 'max_probs': []}, 'BM2': {'max_cost': [], 'max_probs': []}, 'BM3': {'max_cost': [], 'max_probs': []}}, [], []]
